<a href="https://colab.research.google.com/github/randy-ar/gcolab/blob/main/Preprocessing_data_EPA_Chemical_Data_Reporting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [382]:
import pandas as pd
import gdown
import matplotlib.pyplot as plt
import re

# Goals

### Analisis risiko dampak ke lingkungan dan dampak ke manusia yang ditimbulkan oleh bahan kimia

### Output analisis:
1. Choropleth Map: Mengambarkan penggunaan bahan kimia tertinggi dalam suatu state & Mengambarkan jumlah perkerja yang terpapar bahan kimia dalam suatu city
2. Bar Chart: Mengambarkan 10 bahan kimia tertinggi yang tidak di daur ulang yang terdaftar di TSCA (Toxic Substances Control Act)
3. Bar Chart: Mengambarkan 10 sector dengan penggunaan bahan kimia tertinggi

### Manfaat analisis:
1. Dari informasi Choropleth Map tersebut kita dapat melakukan tindakan pencengahan agar bahan kimia tidak memberikan dampak serius ke lingkungan dan pekerja
2. Dari output analisis ke-2 kita dapat memberikan perhatian khusus ke bahan kimia toxic yang tidak di daur ulang
3. Dari output analisis ke-3 kita dapat memberikan perhatian khusus ke sector yang menggunakan bahan kimia paling tinggi

# Read Data CSV

In [383]:
table_consumer_and_use_information = "1dGgcbPnVmOeAP03MMG_jf7FwfQEoTa2P"
table_industrial_processing_and_use_information = "1uS0ucuC24KjENVmqssHiQeL05n4j-LNd"
download_url = "https://docs.google.com/uc?export=download&id="

In [384]:
gdown.download(download_url+table_consumer_and_use_information, 'table_consumer_and_use_information.csv', quiet=False)
gdown.download(download_url+table_industrial_processing_and_use_information, 'table_industrial_processing_and_use_information.csv', quiet=False)

Downloading...
From: https://docs.google.com/uc?export=download&id=1dGgcbPnVmOeAP03MMG_jf7FwfQEoTa2P
To: /content/table_consumer_and_use_information.csv
100%|██████████| 37.0M/37.0M [00:00<00:00, 217MB/s]
Downloading...
From: https://docs.google.com/uc?export=download&id=1uS0ucuC24KjENVmqssHiQeL05n4j-LNd
To: /content/table_industrial_processing_and_use_information.csv
100%|██████████| 48.2M/48.2M [00:00<00:00, 146MB/s]


'table_industrial_processing_and_use_information.csv'

In [385]:
df_consumer = pd.read_csv('table_consumer_and_use_information.csv')
df_industrial = pd.read_csv('table_industrial_processing_and_use_information.csv')

<ipython-input-385-09da472dbfb7>:1: DtypeWarning: Columns (69) have mixed types. Specify dtype option on import or set low_memory=False.
  df_consumer = pd.read_csv('table_consumer_and_use_information.csv')
<ipython-input-385-09da472dbfb7>:2: DtypeWarning: Columns (71) have mixed types. Specify dtype option on import or set low_memory=False.
  df_industrial = pd.read_csv('table_industrial_processing_and_use_information.csv')


In [386]:
df_consumer['DATA CONSUMER'] = 1
df_consumer['DATA INDUSTRIAL'] = 0
df_industrial['DATA CONSUMER'] = 0
df_industrial['DATA INDUSTRIAL'] = 1

In [387]:
df = pd.concat([df_consumer, df_industrial], ignore_index=True)

In [388]:
df.count()

,0
CHEMICAL REPORT ID,121316
CHEMICAL NAME,121316
CHEMICAL ID,121316
CHEMICAL ID W/O DASHES,121316
CHEMICAL ID TYPE,121316
...,...
IND PV PCT,53788
IND SITES CODE,53787
INDUSTRIAL SITES,53787
IND WORKERS CODE,53788


# Preprocessing Data

## CONSUMER DATA

### Mengambil kolom yang dibutuhkan untuk analisa

In [389]:
selected_column = [
    'CHEMICAL ID',
    'CHEMICAL NAME',
    'CHEMICAL ID TYPE',
    'RECYCLED',
    'PHYSICAL FORM(S)',
    '2019 DOMESTIC PV',
    '2019 IMPORT PV',
    '2019 PV',
    '2018 PV',
    '2017 PV',
    '2016 PV',
    'SITE LATITUDE',
    'SITE LONGITUDE',
    'SITE CITY',
    'SITE COUNTY / PARISH',
    'SITE STATE',
    'SITE POSTAL CODE',
    'SITE NAICS CODE 1',
    'SITE NAICS ACTIVITY 1',
    'SITE NAICS CODE 2',
    'SITE NAICS ACTIVITY 2',
    'SITE NAICS CODE 3',
    'SITE NAICS ACTIVITY 3',
    'WORKERS CODE',
    'WORKERS',
    'MAX CONC CODE',
    'MAXIMUM CONCENTRATION',
    'DATA CONSUMER',
    'DATA INDUSTRIAL'
]

In [390]:
df = df[selected_column]
df.isnull().sum()

,0
CHEMICAL ID,187
CHEMICAL NAME,187
CHEMICAL ID TYPE,187
RECYCLED,6575
PHYSICAL FORM(S),12310
2019 DOMESTIC PV,4367
2019 IMPORT PV,4372
2019 PV,4367
2018 PV,348
2017 PV,350


In [391]:
TOTAL_ROWS = df.shape[0]
print(TOTAL_ROWS)

121503


### Mencari jumlah baris yang informasinya di private

In [392]:
cbi_rows = df[df.apply(lambda x: x.astype(str).str.contains('CBI', na=False).any(), axis=1)]
cbi_rows.count()

,0
CHEMICAL ID,68440
CHEMICAL NAME,68440
CHEMICAL ID TYPE,68440
RECYCLED,65955
PHYSICAL FORM(S),62727
2019 DOMESTIC PV,66774
2019 IMPORT PV,66774
2019 PV,66774
2018 PV,68344
2017 PV,68342


In [393]:
TOTAL_CBI_ROWS = cbi_rows.shape[0]
print(TOTAL_CBI_ROWS)

68440


### Menghapus data yang dirahasiakan, data ini sulit dianalisa karena kerahasiannya

In [394]:
# prompt: delete row with CBI Values

# delete rows where any column contains 'CBI'
df = df[~df.apply(lambda x: x.astype(str).str.contains('CBI', na=False).any(), axis=1)]

# verify that CBI rows are removed
cbi_rows_after_removal = df[df.apply(lambda x: x.astype(str).str.contains('CBI', na=False).any(), axis=1)]
print("Number of rows containing 'CBI' after removal:", cbi_rows_after_removal.count().sum())

# show the first few rows of the cleaned dataframe
df.head()

Number of rows containing 'CBI' after removal: 0


,CHEMICAL ID,CHEMICAL NAME,CHEMICAL ID TYPE,RECYCLED,PHYSICAL FORM(S),2019 DOMESTIC PV,2019 IMPORT PV,2019 PV,2018 PV,2017 PV,...,SITE NAICS CODE 2,SITE NAICS ACTIVITY 2,SITE NAICS CODE 3,SITE NAICS ACTIVITY 3,WORKERS CODE,WORKERS,MAX CONC CODE,MAXIMUM CONCENTRATION,DATA CONSUMER,DATA INDUSTRIAL
3,18849,"(Polyisobutenyl)dihydro-2,5-furandione esters ...",Accession Number,No,Liquid,0,"26,564","26,564","27,056","10,321",...,NaN,NaN,NaN,NaN,W3,25 – 49,M2,1% – < 30%,1,0
18,56038-13-2,".alpha.-D-Galactopyranoside, 1,6-dichloro-1,6-...",CASRN,No,Dry Powder,0,"25,187","25,187",495,0,...,NaN,NaN,NaN,NaN,NKRA,Not Known or Reasonably Ascertainable,NKRA,Not Known or Reasonably Ascertainable,1,0
27,57-50-1,".alpha.-D-Glucopyranoside, .beta.-D-fructofura...",CASRN,No,NaN,"2,074,294",0,"2,074,294","1,541,960","2,099,661",...,NaN,NaN,NaN,NaN,W5,100 – 499,M5,90% +,1,0
29,12738-64-6,".alpha.-D-Glucopyranoside, .beta.-D-fructofura...",CASRN,No,NKRA,0,"56,867","56,867","218,213","158,700",...,NaN,NaN,NaN,NaN,W1,< 10,M5,90% +,1,0
30,12738-64-6,".alpha.-D-Glucopyranoside, .beta.-D-fructofura...",CASRN,NaN,NaN,0,0,0,0,"34,780",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0


In [395]:
cbi_rows = df[df.apply(lambda x: x.astype(str).str.contains('CBI', na=False).any(), axis=1)]
cbi_rows.count()

,0
CHEMICAL ID,0
CHEMICAL NAME,0
CHEMICAL ID TYPE,0
RECYCLED,0
PHYSICAL FORM(S),0
2019 DOMESTIC PV,0
2019 IMPORT PV,0
2019 PV,0
2018 PV,0
2017 PV,0


### Hitung data public yang dapat dianalisis

In [396]:
TOTAL_PUBLIC_ROWS = TOTAL_ROWS - TOTAL_CBI_ROWS
print(TOTAL_PUBLIC_ROWS)

53063


In [397]:
# prompt: TOTAL_PUBLIC_ROW is what percent of TOTAL_ROWS

percent_public_rows = (TOTAL_PUBLIC_ROWS / TOTAL_ROWS) * 100
print(f"TOTAL_PUBLIC_ROWS is {percent_public_rows:.2f}% of TOTAL_ROWS")

TOTAL_PUBLIC_ROWS is 43.67% of TOTAL_ROWS


### Data public yang dapat kami analisis adalah 43.67% dari keseluruhan data

In [398]:
df.count()

,0
CHEMICAL ID,52876
CHEMICAL NAME,52876
CHEMICAL ID TYPE,52876
RECYCLED,48973
PHYSICAL FORM(S),46466
2019 DOMESTIC PV,50362
2019 IMPORT PV,50357
2019 PV,50362
2018 PV,52811
2017 PV,52811


### Conversi type data ke Integer dan Float

In [399]:
# prompt: convert this column to integer
#     ['2019 DOMESTIC PV',
#     '2019 IMPORT PV',
#     '2019 PV',
#     '2018 PV',
#     '2017 PV',
#     '2016 PV',]

import pandas as pd
cols_to_convert = [
    '2019 DOMESTIC PV',
    '2019 IMPORT PV',
    '2019 PV',
    '2018 PV',
    '2017 PV',
    '2016 PV',
]

for col in cols_to_convert:
    df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')

In [400]:
# prompt: convert SITE LATITUDE SITE LONGITUDE to float
df['SITE LATITUDE'] = pd.to_numeric(df['SITE LATITUDE'], errors='coerce')
df['SITE LONGITUDE'] = pd.to_numeric(df['SITE LONGITUDE'], errors='coerce')

In [401]:
df.dtypes

,0
CHEMICAL ID,object
CHEMICAL NAME,object
CHEMICAL ID TYPE,object
RECYCLED,object
PHYSICAL FORM(S),object
2019 DOMESTIC PV,Int64
2019 IMPORT PV,Int64
2019 PV,Int64
2018 PV,Int64
2017 PV,Int64


### Mengisi nilai kosong untuk Production Volume

In [402]:
df['2019 DOMESTIC PV'].fillna(int(df['2019 DOMESTIC PV'].mean()), inplace=True)
df['2019 IMPORT PV'].fillna(int(df['2019 IMPORT PV'].mean()), inplace=True)
df['2019 PV'].fillna(int(df['2019 PV'].mean()), inplace=True)
df['2018 PV'].fillna(int(df['2018 PV'].mean()), inplace=True)
df['2017 PV'].fillna(int(df['2017 PV'].mean()), inplace=True)
df['2016 PV'].fillna(int(df['2016 PV'].mean()), inplace=True)

<ipython-input-402-95b081f6e0cb>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['2019 DOMESTIC PV'].fillna(int(df['2019 DOMESTIC PV'].mean()), inplace=True)
<ipython-input-402-95b081f6e0cb>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].metho

### Menghapus nilai chemical id dan chemical name yang kosong

In [403]:
df[['CHEMICAL ID', 'CHEMICAL NAME']].isnull().sum()

,0
CHEMICAL ID,187
CHEMICAL NAME,187


### Memeriksa apakah ada data yang bernilai pada `['CHEMICAL ID', 'CHEMICAL NAME']` yang kosong

In [404]:
# Filter rows where 'CHEMICAL ID' or 'CHEMICAL NAME' is null
null_chemical_info_rows = df[(df['CHEMICAL ID'].isnull()) | (df['CHEMICAL NAME'].isnull())]

# Select the desired columns from the filtered rows
result = null_chemical_info_rows[['CHEMICAL ID TYPE',
                                'RECYCLED',
                                '2019 DOMESTIC PV',
                                '2019 IMPORT PV',
                                '2019 PV',
                                '2018 PV',
                                '2017 PV',
                                '2016 PV',
                                'SITE LATITUDE',
                                'SITE LONGITUDE',
                                'SITE CITY',
                                'SITE COUNTY / PARISH',
                                'SITE STATE',
                                'SITE POSTAL CODE',
                                'SITE NAICS CODE 1',
                                'SITE NAICS ACTIVITY 1',
                                'SITE NAICS CODE 2',
                                'SITE NAICS ACTIVITY 2',
                                'SITE NAICS CODE 3',
                                'SITE NAICS ACTIVITY 3',]]

# Print the resulting DataFrame
result

,CHEMICAL ID TYPE,RECYCLED,2019 DOMESTIC PV,2019 IMPORT PV,2019 PV,2018 PV,2017 PV,2016 PV,SITE LATITUDE,SITE LONGITUDE,SITE CITY,SITE COUNTY / PARISH,SITE STATE,SITE POSTAL CODE,SITE NAICS CODE 1,SITE NAICS ACTIVITY 1,SITE NAICS CODE 2,SITE NAICS ACTIVITY 2,SITE NAICS CODE 3,SITE NAICS ACTIVITY 3
56000,NaN,NaN,2,7,48,30,27,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56001,NaN,NaN,2,7,48,30,27,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56002,NaN,NaN,2,7,48,30,27,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56003,NaN,NaN,2,7,48,30,27,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56004,NaN,NaN,2,7,48,30,27,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121498,NaN,NaN,2,7,48,30,27,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
121499,NaN,NaN,2,7,48,30,27,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
121500,NaN,NaN,2,7,48,30,27,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
121501,NaN,NaN,2,7,48,30,27,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [405]:
# prompt: i want to delete row who has null chemical id or chemical name

# Delete rows where 'CHEMICAL ID' or 'CHEMICAL NAME' is null
df.dropna(subset=['CHEMICAL ID', 'CHEMICAL NAME'], inplace=True)

# Check for null values after dropping
df[['CHEMICAL ID', 'CHEMICAL NAME']].isnull().sum()

,0
CHEMICAL ID,0
CHEMICAL NAME,0


In [406]:
df.isnull().sum()

,0
CHEMICAL ID,0
CHEMICAL NAME,0
CHEMICAL ID TYPE,0
RECYCLED,3903
PHYSICAL FORM(S),6410
2019 DOMESTIC PV,0
2019 IMPORT PV,0
2019 PV,0
2018 PV,0
2017 PV,0


### Mengubah format nama tempat menjadi seragam menggunakan UPPERCASE

In [407]:
# prompt: i want to format SITE CITY, SITE COUNTY / PARISH, and  SITE STATE to uppercase

df['SITE CITY'] = df['SITE CITY'].str.upper()
df['SITE COUNTY / PARISH'] = df['SITE COUNTY / PARISH'].str.upper()
df['SITE STATE'] = df['SITE STATE'].str.upper()
df['SITE POSTAL CODE'] = df['SITE POSTAL CODE'].str.upper()

In [408]:
# prompt: i want to know state city who has site latitude and site longtitude null or 0, i want unique value of that city list

# Filter rows where SITE LATITUDE or SITE LONGITUDE is null or 0
null_zero_lat_lon = df[(df['SITE LATITUDE'].isnull()) |
                              (df['SITE LATITUDE'] == 0) |
                              (df['SITE LONGITUDE'].isnull()) |
                              (df['SITE LONGITUDE'] == 0)]

# Get the unique list of cities from the filtered rows
cities_with_null_zero_lat_lon = null_zero_lat_lon['SITE CITY'].unique()
states_with_null_zero_lat_lon = null_zero_lat_lon['SITE STATE'].unique()

# Print the unique list of cities
print("Cities with null or 0 Site Latitude or Site Longitude:")
print(cities_with_null_zero_lat_lon)
print(states_with_null_zero_lat_lon)


Cities with null or 0 Site Latitude or Site Longitude:
['HENDERSON' 'NEW CASTLE' 'OKLAHOMA CITY' 'HOUSTON' 'TRENTON'
 'FARMINGTON HILLS' 'PHOENIX' 'WITHHELD' 'SADDLE BROOK' 'NEWARK'
 'GEORGETOWN' 'MILL HALL' 'BAYTOWN' 'ROCKET CENTER' 'PASADENA'
 'LAKE CHARLES' 'EAST WINDSOR' 'WEST POINT' 'NEW YORK' 'ROME'
 'GOLDEN MEADOW' 'FREEPORT' 'SHELTON' 'LOUISVILLE' 'NASHVILLE' 'KOTZEBUE'
 'MCCARRAN' 'SOUTH DEERFIELD' 'BROOKFIELD' 'BRIDGEWATER' 'SONORA'
 'GOOSE CREEK' 'NEW KENSINGTON' 'IMPERIAL' 'MARYSVILLE' 'TROY' 'FRIENDLY'
 'CHICAGO HEIGHTS' 'DORADO' 'BROOKSVILLE' 'RAPID CITY' 'THOMASTON'
 'PAULDING' 'SELLERSBURG' 'STOCKTON' 'MARYNEAL' 'WASHINGTON'
 'OAKBROOK TERRACE' 'OAKLAND' 'CORTLAND' 'IRVING' 'POST FALLS'
 'MOUNDSVILLE' 'BRILLIANT' 'BEULAH' 'SCHOFIELD BARRACKS' 'PORTLAND'
 'ROCK SPRINGS' 'WHEATFIELD' 'LATHROP' 'POCATELLO' 'FORT MADISON'
 'LAVERGNE' 'MAIDSVILLE' 'DIBOLL' 'PENSACOLA' 'PANAMA' 'PINEVILLE'
 'SUNNYSIDE' 'ROOPVILLE' 'COLSTRIP' 'BATTLE MOUNTAIN' 'WEST CHICAGO'
 'SAINT GABRIEL' '

### Memeriksa kota dengan nilai state `NaN`

In [409]:
# prompt: i want to know city name, lat, long, and postal code where state is nan

nan_state_info = df[df['SITE STATE'].isnull()][['SITE CITY', 'SITE LATITUDE', 'SITE LONGITUDE', 'SITE POSTAL CODE']]
print("Information for entries where SITE STATE is NaN:")
nan_state_info

Information for entries where SITE STATE is NaN:


,SITE CITY,SITE LATITUDE,SITE LONGITUDE,SITE POSTAL CODE
23872,"BURRA, SA 5417",0.0,0.0,NaN
23923,"BURRA, SA 5417",0.0,0.0,NaN
83224,"BURRA, SA 5417",0.0,0.0,NaN
83225,"BURRA, SA 5417",0.0,0.0,NaN
83226,"BURRA, SA 5417",0.0,0.0,NaN
83227,"BURRA, SA 5417",0.0,0.0,NaN
83305,"BURRA, SA 5417",0.0,0.0,NaN


### Menghapus baris dari kota BURRA, SA 5417, karena kota tersebut bukan ada di wilayah Amerika Serikat

In [410]:
# prompt: i want to delete row where state is nan

df.dropna(subset=['SITE STATE'], inplace=True)
df.isnull().sum()

,0
CHEMICAL ID,0
CHEMICAL NAME,0
CHEMICAL ID TYPE,0
RECYCLED,3903
PHYSICAL FORM(S),6410
2019 DOMESTIC PV,0
2019 IMPORT PV,0
2019 PV,0
2018 PV,0
2017 PV,0


### Memeriksa nama tempat yang dirahasiakan

In [411]:
# prompt: i want to know city name, lat, long, and postal code where state is WITHHELD

df[df['SITE STATE'] == 'WITHHELD'][['SITE CITY', 'SITE LATITUDE', 'SITE LONGITUDE', 'SITE POSTAL CODE']]

,SITE CITY,SITE LATITUDE,SITE LONGITUDE,SITE POSTAL CODE
980,WITHHELD,NaN,NaN,WITHHELD
981,WITHHELD,NaN,NaN,WITHHELD
982,WITHHELD,NaN,NaN,WITHHELD
1036,WITHHELD,NaN,NaN,WITHHELD
1262,WITHHELD,NaN,NaN,WITHHELD
...,...,...,...,...
120393,WITHHELD,NaN,NaN,WITHHELD
120394,WITHHELD,NaN,NaN,WITHHELD
120395,WITHHELD,NaN,NaN,WITHHELD
120396,WITHHELD,NaN,NaN,WITHHELD


### Membuat Informasi Lat Long untuk di petakan pada nilai kosong yang memiliki nama kota

In [412]:
long_lat_missing_city = {
    'HENDERSON': [37.842777, -87.587222],
    'NEW CASTLE': [39.679558, -75.599933],
    'OKLAHOMA CITY': [35.467560, -97.516428],
    'HOUSTON': [29.760427, -95.369803],
    'TRENTON': [40.220109, -74.766861],
    'FARMINGTON HILLS': [42.482811, -83.376884],
    'PHOENIX': [33.448377, -112.074037],
    'SADDLE BROOK': [40.916766, -74.073479],
    'NEWARK': [40.735657, -74.172366],
    'GEORGETOWN': [38.718693, -75.122687],
    'MILL HALL': [41.135246, -77.464731],
    'BAYTOWN': [29.749947, -95.031326],
    'ROCKET CENTER': [39.549266, -78.895066],
    'PASADENA': [29.610508, -95.207705],
    'LAKE CHARLES': [30.224021, -93.217384],
    'EAST WINDSOR': [40.297444, -74.526278],
    'WEST POINT': [33.606775, -88.647547],
    'NEW YORK': [40.7128, -74.0060],
    'ROME': [34.257038, -85.164673],
    'GOLDEN MEADOW': [29.387167, -90.257545],
    'FREEPORT': [40.658717, -73.582631],
    'SHELTON': [41.325659, -73.136224],
    'LOUISVILLE': [38.252665, -85.758456],
    'NASHVILLE': [36.162664, -86.781601],
    'KOTZEBUE': [66.898056, -162.585833],
    'MCCARRAN': [39.545899, -119.569485],
    'SOUTH DEERFIELD': [42.493988, -72.607590],
    'BROOKFIELD': [43.059458, -88.093144],
    'BRIDGEWATER': [40.575654, -74.586616],
    'SONORA': [37.984638, -120.383526],
    'GOOSE CREEK': [33.003223, -80.034252],
    'NEW KENSINGTON': [40.573950, -79.761719],
    'IMPERIAL': [32.846430, -115.564440],
    'MARYSVILLE': [39.143789, -121.591901],
    'TROY': [42.728333, -73.692500],
    'FRIENDLY': [39.564522, -81.047067],
    'CHICAGO HEIGHTS': [41.503923, -87.641716],
    'DORADO': [18.455209, -66.273775],
    'BROOKSVILLE': [28.555556, -82.395833],
    'RAPID CITY': [44.080556, -103.227222],
    'THOMASTON': [32.890691, -84.288544],
    'PAULDING': [41.144497, -84.582458],
    'SELLERSBURG': [38.411171, -85.760803],
    'STOCKTON': [37.957702, -121.290780],
    'MARYNEAL': [32.253056, -101.442222],
    'WASHINGTON': [40.173685, -80.245065],
    'OAKBROOK TERRACE': [41.854477, -87.954784],
    'OAKLAND': [37.804363, -122.271113],
    'CORTLAND': [42.599793, -76.177264],
    'IRVING': [32.814018, -96.948895],
    'POST FALLS': [47.715732, -116.953767],
    'MOUNDSVILLE': [39.914246, -80.749806],
    'BRILLIANT': [40.354238, -80.607028],
    'BEULAH': [47.241389, -101.777778],
    'SCHOFIELD BARRACKS': [21.4925, -158.058056],
    'PORTLAND': [45.523062, -122.676482],
    'ROCK SPRINGS': [41.591079, -109.202353],
    'WHEATFIELD': [41.228122, -87.112799],
    'LATHROP': [37.795908, -121.240502],
    'POCATELLO': [42.871032, -112.433220],
    'FORT MADISON': [40.627257, -91.317926],
    'LAVERGNE': [35.945203, -86.568600],
    'MAIDSVILLE': [39.733979, -79.996172],
    'DIBOLL': [31.189623, -94.795213],
    'PENSACOLA': [30.421309, -87.216912],
    'PANAMA': [39.068930, -89.431200],
    'PINEVILLE': [31.323516, -92.433470],
    'SUNNYSIDE': [46.333189, -120.007550],
    'ROOPVILLE': [33.565116, -85.122170],
    'COLSTRIP': [45.892789, -106.613360],
    'BATTLE MOUNTAIN': [40.638515, -116.909538],
    'WEST CHICAGO': [41.897258, -88.209800],
    'SAINT GABRIEL': [30.298254, -91.077051],
    'CALHOUN, GORDON': [34.502035, -84.945070], # Assuming this refers to Gordon, Calhoun County, GA
    'UNION': [37.766768, -80.540356],
    'POINT PLEASANT': [38.847585, -82.128768],
    'CLEVELAND': [41.499320, -81.694361],
    'SAHUARITA': [31.954546, -111.002873],
    'ALABASTER': [33.242898, -86.820847],
    'TICONDEROGA': [43.834458, -73.415053],
    'DUVALL': [47.781216, -121.977067],
    'MCALESTER': [34.927038, -95.770267],
    'COLUMBUS': [39.961176, -82.998794],
    'CEDAR SPRINGS': [43.208649, -85.556419],
    'PARSIPPANY': [40.854823, -74.407659],
    'WICKLIFFE': [41.602551, -81.470409],
    'FLORENCE': [38.006940, -84.620000],
    'FRIESLAND': [43.559439, -89.043729],
    'CHRISTIANSTED': [17.747978, -64.703487],
    'MANCHESTER': [42.993056, -71.464167],
    'SAGINAW': [43.419470, -83.950807],
    'WELLESLEY HILLS': [42.296541, -71.258814],
    'LATROBE': [40.327568, -79.395039],
    'BALDWIN': [33.090956, -83.479590],
    'GHENT': [42.348692, -73.619010],
    'WOODS CROSS': [40.893116, -111.916053],
    'CROSSETT': [33.102334, -91.996245],
    'CANBY': [45.260124, -122.693149],
    'ANCHORAGE': [61.218056, -149.900278],
    'MARIETTA': [33.952607, -84.549933],
    'CAMDEN CITY': [39.945833, -75.101111],
    'PUEBLO': [38.254477, -104.609100],
    'BAY CITY': [28.980269, -96.146919],
    'HELM': [36.467451, -120.151259],
    'MCINTOSH': [44.757739, -95.955437],
    'LAWRENCE': [38.971667, -95.235278],
    'MULBERRY': [27.904467, -81.994537],
    'GARRETSON': [43.702758, -96.486745],
    'FORT LUPTON': [40.198319, -104.809971],
    'GREELEY': [40.426578, -104.709968],
    'SILVER BOW': [46.037166, -112.569472],
    'ELMENDORF': [29.231061, -98.411689],
    'CARTERSVILLE': [34.166667, -84.806389],
    'PLYMOUTH': [41.958333, -70.667500],
    'MONTICELLO': [40.758368, -86.764506],
    'LYONS': [40.231221, -98.666141],
    'SCHUYLER': [41.442222, -96.903056],
    'SANTA ANA': [33.745484, -117.867623]
}

In [413]:
# prompt: I want to fill null or 0 value in SITE LATITUDE and SITE LONGITUDE with long_lat_missing_city

import pandas as pd
# Iterate through the DataFrame and fill null/0 values
for index, row in df.iterrows():
    if (pd.isnull(row['SITE LATITUDE']) or row['SITE LATITUDE'] == 0 or
        pd.isnull(row['SITE LONGITUDE']) or row['SITE LONGITUDE'] == 0):
        city = row['SITE CITY']
        if city in long_lat_missing_city:
            df.loc[index, 'SITE LATITUDE'] = long_lat_missing_city[city][0]
            df.loc[index, 'SITE LONGITUDE'] = long_lat_missing_city[city][1]

# Verify the changes
print("\nNull/0 values in SITE LATITUDE and SITE LONGITUDE after filling:")
print(df[(df['SITE LATITUDE'].isnull()) |
                (df['SITE LATITUDE'] == 0) |
                (df['SITE LONGITUDE'].isnull()) |
                (df['SITE LONGITUDE'] == 0)][['SITE CITY', 'SITE LATITUDE', 'SITE LONGITUDE']].count())


Null/0 values in SITE LATITUDE and SITE LONGITUDE after filling:
SITE CITY         132
SITE LATITUDE       0
SITE LONGITUDE      0
dtype: int64


In [414]:
df.isnull().sum()

,0
CHEMICAL ID,0
CHEMICAL NAME,0
CHEMICAL ID TYPE,0
RECYCLED,3903
PHYSICAL FORM(S),6410
2019 DOMESTIC PV,0
2019 IMPORT PV,0
2019 PV,0
2018 PV,0
2017 PV,0


### Mengisi nilai lat long `[-1, -1]` untuk informasi wilayah yang dirahasiakan

In [415]:
# prompt: i want to fill lat long where value is NaN with [-1, -1]

df['SITE LATITUDE'].fillna(-1, inplace=True)
df['SITE LONGITUDE'].fillna(-1, inplace=True)

<ipython-input-415-944261fbb56a>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['SITE LATITUDE'].fillna(-1, inplace=True)
<ipython-input-415-944261fbb56a>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 

### Mecari nilai NKRA atau Not Known or Reasonably Ascertainable

In [416]:
# prompt: i want to find column where has value 'Not Known or Reasonably Ascertainable'

for col in df.columns:
    if (df[col] == 'Not Known or Reasonably Ascertainable').any():
        print(f"Column '{col}' contains 'Not Known or Reasonably Ascertainable'")

Column 'WORKERS' contains 'Not Known or Reasonably Ascertainable'
Column 'MAXIMUM CONCENTRATION' contains 'Not Known or Reasonably Ascertainable'


In [417]:
# prompt: i want to replace 'Not Known or Reasonably Ascertainable' to 'NKRA'

df.replace('Not Known or Reasonably Ascertainable', 'NKRA', inplace=True)

In [418]:
# prompt: i want to find WORKERS value where WORKERS CODE value is NKRA and WORKER value is not NKRA

workers_nkra_worker_not_nkra = df[(df['WORKERS CODE'] == 'NKRA') & (df['WORKERS'] != 'NKRA')]

print("Entries where 'WORKERS CODE' is 'NKRA' and 'WORKERS' is not 'NKRA':")
print(workers_nkra_worker_not_nkra[['WORKERS CODE', 'WORKERS']])

Entries where 'WORKERS CODE' is 'NKRA' and 'WORKERS' is not 'NKRA':
Empty DataFrame
Columns: [WORKERS CODE, WORKERS]
Index: []


### Membuat kolom workers median

In [419]:
# prompt: drop workers median column if exist

if 'WORKERS median' in df.columns:
    df = df.drop('WORKERS median', axis=1)

print("Columns after potentially dropping 'WORKERS median':")
df.columns

Columns after potentially dropping 'WORKERS median':


Index(['CHEMICAL ID', 'CHEMICAL NAME', 'CHEMICAL ID TYPE', 'RECYCLED',
       'PHYSICAL FORM(S)', '2019 DOMESTIC PV', '2019 IMPORT PV', '2019 PV',
       '2018 PV', '2017 PV', '2016 PV', 'SITE LATITUDE', 'SITE LONGITUDE',
       'SITE CITY', 'SITE COUNTY / PARISH', 'SITE STATE', 'SITE POSTAL CODE',
       'SITE NAICS CODE 1', 'SITE NAICS ACTIVITY 1', 'SITE NAICS CODE 2',
       'SITE NAICS ACTIVITY 2', 'SITE NAICS CODE 3', 'SITE NAICS ACTIVITY 3',
       'WORKERS CODE', 'WORKERS', 'MAX CONC CODE', 'MAXIMUM CONCENTRATION',
       'DATA CONSUMER', 'DATA INDUSTRIAL'],
      dtype='object')

In [420]:
# prompt: create fuction to calculate workers median: 1. split string with space or any white space. 2. filter array, take only integer value 3. return sum(list)/len(list)
def calculate_workers_median(workers_string):
    """
    Calculates the median of worker values from a string.

    1. Splits the input string by whitespace.
    2. Filters the resulting array to include only integer values.
    3. Calculates and returns the median of the integer values.
       Returns None if no valid integers are found.
    """
    if pd.isnull(workers_string) or workers_string == 'NKRA':
        return None

    parts = workers_string.split()
    integer_workers = [int(part) for part in parts if part.isdigit()]

    if not integer_workers:
        return None

    integer_workers.sort()
    n = len(integer_workers)
    return int((sum(integer_workers)+1)//len(integer_workers))

df['WORKERS MEDIAN'] = df['WORKERS'].apply(calculate_workers_median)

# Display the first few rows with the new column
print(df[['WORKERS', 'WORKERS CODE', 'WORKERS MEDIAN']].head())

# Check for null values in the new column
print("\nNull values in WORKERS MEDIAN after calculation:")

      WORKERS WORKERS CODE  WORKERS MEDIAN
3     25 – 49           W3            37.0
18       NKRA         NKRA             NaN
27  100 – 499           W5           300.0
29       < 10           W1            11.0
30        NaN          NaN             NaN

Null values in WORKERS MEDIAN after calculation:


In [421]:
df.isnull().sum()

,0
CHEMICAL ID,0
CHEMICAL NAME,0
CHEMICAL ID TYPE,0
RECYCLED,3903
PHYSICAL FORM(S),6410
2019 DOMESTIC PV,0
2019 IMPORT PV,0
2019 PV,0
2018 PV,0
2017 PV,0


### Mengisi nilai NaN pada Workers Median berdasarkan rata-rata di kelompok kota atau nilai 0 jika tidak ada informasi wilayah

In [422]:
# prompt: fill NaN value in workers median with modes in SITE CITY groups, if theres no SITE CITY information fill it with 0

# Fill NaN values in 'WORKERS MEDIAN' with the mode of the 'WORKERS MEDIAN'
# within each group of 'SITE CITY' and 'SITE STATE'.
# If a group is empty or has no mode, fill with 0.
df['WORKERS MEDIAN'] = df.groupby(['SITE CITY', 'SITE STATE'])['WORKERS MEDIAN'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 0))

# Verify that there are no more NaN values in 'WORKERS MEDIAN'
print("\nNull values in WORKERS MEDIAN after filling:")
print(df['WORKERS MEDIAN'].isnull().sum())



Null values in WORKERS MEDIAN after filling:
0


In [423]:
df.isnull().sum()

,0
CHEMICAL ID,0
CHEMICAL NAME,0
CHEMICAL ID TYPE,0
RECYCLED,3903
PHYSICAL FORM(S),6410
2019 DOMESTIC PV,0
2019 IMPORT PV,0
2019 PV,0
2018 PV,0
2017 PV,0


In [424]:
# Fill any remaining NaN values in 'WORKERS MEDIAN' (where there was no SITE CITY information) with 0
df['WORKERS MEDIAN'].fillna(0, inplace=True)

<ipython-input-424-8186ca7ffd99>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['WORKERS MEDIAN'].fillna(0, inplace=True)


In [425]:
df.isnull().sum()

,0
CHEMICAL ID,0
CHEMICAL NAME,0
CHEMICAL ID TYPE,0
RECYCLED,3903
PHYSICAL FORM(S),6410
2019 DOMESTIC PV,0
2019 IMPORT PV,0
2019 PV,0
2018 PV,0
2017 PV,0


### Kolom Chemical Form(s)

In [426]:
df.isnull().sum()

,0
CHEMICAL ID,0
CHEMICAL NAME,0
CHEMICAL ID TYPE,0
RECYCLED,3903
PHYSICAL FORM(S),6410
2019 DOMESTIC PV,0
2019 IMPORT PV,0
2019 PV,0
2018 PV,0
2017 PV,0


In [427]:
# prompt: i want unique value of chemical forms

unique_chemical_forms = df['PHYSICAL FORM(S)'].unique()
print("Unique Chemical Forms:")
unique_chemical_forms

Unique Chemical Forms:


array(['Liquid', 'Dry Powder', nan, 'NKRA', 'Water or Solvent Wet Solid',
       'Pellets or Large Crystals', 'Pellets or Large Crystals, Liquid',
       'Other Solid', 'Liquid, Other Solid', 'Dry Powder, Liquid',
       'Dry Powder, Pellets or Large Crystals',
       'Dry Powder, Liquid, Other Solid', 'Gas or Vapor',
       'Gas or Vapor, Liquid, Other Solid',
       'Water or Solvent Wet Solid, Other Solid',
       'Dry Powder, Other Solid',
       'Pellets or Large Crystals, Other Solid',
       'Dry Powder, Water or Solvent Wet Solid',
       'Water or Solvent Wet Solid, Liquid',
       'Dry Powder, Pellets or Large Crystals, Other Solid',
       'Dry Powder, Pellets or Large Crystals, Water or Solvent Wet Solid, Liquid',
       'Gas or Vapor, Other Solid', 'Gas or Vapor, Liquid',
       'Dry Powder, Pellets or Large Crystals, Water or Solvent Wet Solid',
       'Liquid, NKRA',
       'Pellets or Large Crystals, Water or Solvent Wet Solid',
       'Pellets or Large Crystals, Liquid

### Check Kolom Maximum Concentration

In [428]:
df[['MAX CONC CODE','MAXIMUM CONCENTRATION']].isnull().sum()

,0
MAX CONC CODE,3905
MAXIMUM CONCENTRATION,3905


In [429]:
df['MAX CONC CODE'].unique()

array(['M2', 'NKRA', 'M5', nan, 'M1', 'M3', 'M4'], dtype=object)

In [430]:
df['MAXIMUM CONCENTRATION'].unique()

array(['1% – < 30%', 'NKRA', '90% +', nan, '< 1%', '30% – < 60%',
       '60% – < 90%'], dtype=object)

In [431]:
df_nkra_max_conc = df[
    (
        (df['MAX CONC CODE'] == 'NKRA') |
        (df['MAX CONC CODE'] == None)
    ) &
    (df['CHEMICAL ID TYPE'] == 'Accession Number')
    ]
df_nkra_max_conc[['MAX CONC CODE','MAXIMUM CONCENTRATION']].count()

,0
MAX CONC CODE,10
MAXIMUM CONCENTRATION,10


### Mengisi Nilai `Unknown` ke semua kolom yang nilainya kosong

In [432]:
for col in selected_column:
    df[col].fillna('Unknown', inplace=True)

<ipython-input-432-a243da23ac13>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna('Unknown', inplace=True)


In [433]:
df.isnull().sum()

,0
CHEMICAL ID,0
CHEMICAL NAME,0
CHEMICAL ID TYPE,0
RECYCLED,0
PHYSICAL FORM(S),0
2019 DOMESTIC PV,0
2019 IMPORT PV,0
2019 PV,0
2018 PV,0
2017 PV,0


### Mempersingkat nama bahan kimia

In [434]:
def summarize_chemical_name(name):
    # 1. Menghapus informasi dalam kurung (misalnya (1:1), (PROVISIONAL))
    name = re.sub(r'\s*\([^)]*\)', '', name)

    # 2. Menghapus simbol seperti .alpha.-, .beta.-, .fwdarw.
    name = re.sub(r'\.(alpha|beta)\.-', '', name)
    name = name.replace('.fwdarw.', '>') # Mengganti panah dengan simbol lebih ringkas
    name = name.replace('rel-', '') # Menghapus 'rel-'
    name = name.replace('compd. with', 'compd.') # Meringkas 'compd. with'

    # 3. Mengganti "reaction products with" dengan "Rxn Prod."
    name = name.replace('reaction products with', 'Rxn Prod. with')
    name = name.replace('reaction product with', 'Rxn Prod. with')
    name = name.replace('reaction product', 'Rxn Prod.')
    name = name.replace('reaction products', 'Rxn Prod.')

    # 4. Meringkas beberapa kata umum
    name = name.replace('carboxylic acid', 'CA')
    name = name.replace('dihydroxyphenyl', 'DHPO')
    name = name.replace('dimethylethyl', 't-butyl') # Mengganti '1,1-dimethylethyl' dengan 't-butyl'
    name = name.replace('dimethyl', 'diMe')
    name = name.replace('hexamethyl', 'hexaMe')
    name = name.replace('tetramethyl', 'tetraMe')
    name = name.replace('trimethyl', 'triMe')
    name = name.replace('triethylamine', 'TEA')
    name = name.replace('polyethyleneamines', 'PEA')
    name = name.replace('polyisobutenyl', 'PIB')
    name = name.replace('hydroxyphenyl', 'HPO')
    name = name.replace('phenylmethyl', 'Bn') # Benzyl
    name = name.replace('ethylhexyl', 'EH')
    name = name.replace('hydroxyethyl', 'HE')
    name = name.replace('octadecenyl', 'oleyl')
    name = name.replace('octadecanoate', 'stearate')
    name = name.replace('propanediol', 'PD')
    name = name.replace('ethanediyl', 'EDA')
    name = name.replace('phosphate', 'PO4')
    name = name.replace('hydrochloride', 'HCl')
    name = name.replace('sodium salt', 'Na salt')
    name = name.replace('potassium salt', 'K salt')
    name = name.replace('lithium salt', 'Li salt')
    name = name.replace('ammonium salt', 'NH4 salt')
    name = name.replace('magnesium salt', 'Mg salt')
    name = name.replace('calcium salt', 'Ca salt')
    name = name.replace('sulfonic acid', 'SA')
    name = name.replace('dihydro-2,5-furandione', 'furandione') # Meringkas nama inti

    # 5. Menghapus koma berlebihan dan spasi ganda
    name = re.sub(r',+', ',', name) # Ganti multiple commas dengan single comma
    name = re.sub(r',\s*$', '', name) # Hapus koma di akhir
    name = re.sub(r'^\s*,\s*', '', name) # Hapus koma di awal
    name = re.sub(r'\s{2,}', ' ', name) # Ganti multiple spaces dengan single space
    name = name.strip() # Hapus spasi di awal/akhir

    # Hapus semua simbol dan angka
    name = re.sub(r'[^a-zA-Z\s]', '', name)
    # Hapus spasi berlebih
    name = re.sub(r'\s+', ' ', name).strip()

    # Default fallback: hapus detail yang sangat spesifik
    name = re.sub(r'^\d{1,3}(,\d{1,3})*-', '', name) # Hapus angka awal
    name = re.sub(r'\(\d:\?\)|missing number|\(\d:\d\)', '', name) # Hapus rasio
    name = re.sub(r'\(\w{2}-\d{1,2}-\d{1,2}-?\)', '', name) # Hapus kode identifikasi
    name = re.sub(r'-\.kappa\.\w(\,\.kappa\.\w)?', '', name) # Hapus notasi kappa
    name = name.replace('(PROVISIONAL)', '').strip()
    name = name.replace('derivs.', '').strip()
    name = name.replace('compd. with', '').strip()
    name = name.replace('reaction products with', '').strip()
    name = re.sub(r'\s+', ' ', name).strip() # Hapus spasi berlebih

    # Ambil beberapa kata pertama jika masih terlalu panjang, atau biarkan begitu
    if len(name.split()) > 3:
        return ' '.join(name.split()[:3]) # Ambil 3 kata pertama
    return name.title() # Kapitalisasi setiap kata

In [435]:
# prompt: i want to know unique value of chemical name

# Get the unique values of 'CHEMICAL NAME'
unique_chemical_names = df['CHEMICAL NAME'].unique()

# Print the unique chemical names
print("Unique Chemical Names:")
print(len(unique_chemical_names))

Unique Chemical Names:
4658


In [436]:
# prompt: i want to know unique value of chemical id

print("Unique Chemical NAMEs:")
for name in df['CHEMICAL NAME'].unique():
    print(name)

Unique Chemical NAMEs:
(Polyisobutenyl)dihydro-2,5-furandione esters with polyhydroxylalkane
.alpha.-D-Galactopyranoside, 1,6-dichloro-1,6-dideoxy-.beta.-D-fructofuranosyl 4-chloro-4-deoxy-
.alpha.-D-Glucopyranoside, .beta.-D-fructofuranosyl
.alpha.-D-Glucopyranoside, .beta.-D-fructofuranosyl, benzoate
.beta.-Alanine, N-(2-carboxyethyl)-N-(2-ethylhexyl)-, sodium salt (1:1)
.beta.-Alanine, N-(2-carboxyethyl)-N-dodecyl-, sodium salt (1:1)
.beta.-Alanine, N-(2-carboxyethyl)-N-dodecyl-, sodium salt (1:2)
.beta.-Cyclodextrin, 2-hydroxypropyl ethers
[1,1'-Biphenyl]-2,2'-diol, 3,3',5,5'-tetrakis(1,1-dimethylethyl)-
[1,1'-Biphenyl]-3,3',4,4'-tetramine
[1,1'-Biphenyl]-4,4'-diamine, 3,3'-dichloro-, hydrochloride (1:2)
[1,1'-Biphenyl]-4,4'-diamine, 3,3'-dimethoxy-, hydrochloride (1:2)
[1,1'-Biphenyl]-4,4'-diol
[1,1'-Biphenyl]-4-carboxamide, 4',4'''-(1,2-diazenediyl)bis[N-(9,10-dihydro-9,10-dioxo-1-anthracenyl)-
[1,2,4]Triazolo[1,5-a]pyrimidin-2-amine, 5,7-dimethoxy-
[2,2'-Bi-1H-indole]-3,3'-diol,

In [437]:
df['CHEMICAL NAME SUMMARY'] = df['CHEMICAL NAME'].apply(summarize_chemical_name)

In [438]:
df['CHEMICAL NAME SUMMARY'].tail(40)

,CHEMICAL NAME SUMMARY
121219,Zirconium
121220,Zirconium
121221,Zirconium
121222,Zirconium
121223,Zirconium
121233,Zirconium Oxide
121246,Zirconium Oxide
121252,Zirconium Oxide
121253,Zirconium Oxide
121254,Zirconium Oxide


### Mempersingkat nama sector

In [439]:
# Get the unique values of 'SITE NAICS CODE 1'
unique_sector_name = df['SITE NAICS CODE 1'].unique()

# Print the unique chemical names
print("Unique Chemical Names:")
print(len(unique_sector_name))

Unique Chemical Names:
268


In [440]:
unique_sector_name

array(['324191 Petroleum Lubricating Oil And Grease Manufacturing',
       '424690 Other Chemical And Allied Products Merchant Wholesalers',
       '111930 Sugarcane Farming',
       '325199 All Other Basic Organic Chemical Manufacturing',
       '325611 Soap And Other Detergent Manufacturing',
       '424910 Farm Supplies Merchant Wholesalers',
       '339942 Lead Pencil And Art Good Manufacturing',
       '325130 Synthetic Dye And Pigment Manufacturing',
       '325998 All Other Miscellaneous Chemical Product And Preparation Manufacturing',
       '324110 Petroleum Refineries',
       '325110 Petrochemical Manufacturing',
       '336999 All Other Transportation Equipment Manufacturing',
       '325188 All Other Basic Inorganic Chemical Manufacturing',
       '325613 Surface Active Agent Manufacturing',
       '453998 All Other Miscellaneous Store Retailers (Except Tobacco Stores)',
       '325180 Other Basic Inorganic Chemical Manufacturing',
       '422690 Other Chemical And Allied 

In [441]:
# prompt: i want to separate id in sector name naics 1

def extract_id_from_naics(naics_string):
  """
  Extracts the numeric ID from a NAICS string.

  Assumes the format is 'ID - Sector Name'.
  Returns the ID as a string, or the original string if the format doesn't match.
  """
  if isinstance(naics_string, str):
    match = re.match(r'(\d+) ', naics_string)
    if match:
      return match.group(1)
  return naics_string

df['SITE NAICS ID 1'] = df['SITE NAICS CODE 1'].apply(extract_id_from_naics)

In [442]:
print(df[['SITE NAICS CODE 1', 'SITE NAICS ID 1']].head())
print("\nUnique NAICS IDs 1 after extraction:")
print(df['SITE NAICS ID 1'].unique())

                                    SITE NAICS CODE 1 SITE NAICS ID 1
3   324191 Petroleum Lubricating Oil And Grease Ma...          324191
18  424690 Other Chemical And Allied Products Merc...          424690
27                           111930 Sugarcane Farming          111930
29  424690 Other Chemical And Allied Products Merc...          424690
30  325199 All Other Basic Organic Chemical Manufa...          325199

Unique NAICS IDs 1 after extraction:
['324191' '424690' '111930' '325199' '325611' '424910' '339942' '325130'
 '325998' '324110' '325110' '336999' '325188' '325613' '453998' '325180'
 '422690' '325312' '325520' '213112' '325193' '311225' '551114' '325411'
 '311224' '333318' '325910' '311222' '423430' '331420' '336111' '424120'
 '325211' '325920' '325991' '424610' '325510' '326220' '325194' '325320'
 '325212' 'Withheld' '423320' '311930' '326192' '331410' '423840' '327120'
 '324199' '325413' '423110' '325620' '326211' '336340' '335110' '336510'
 '336415' '423910' '331421' '

In [443]:
# prompt: create function to extract sector name form SITE NAICS ID 1

def extract_sector_name_from_naics(naics_string):
  """
  Extracts the sector name from a NAICS string.

  Assumes the format is 'ID - Sector Name'.
  Returns the sector name as a string, or the original string if the format doesn't match.
  """
  if isinstance(naics_string, str):
    parts = naics_string.split(' ', 1)
    if len(parts) == 2:
      return parts[1].strip()
  return naics_string

df['SITE NAICS NAME 1'] = df['SITE NAICS CODE 1'].apply(extract_sector_name_from_naics)

In [444]:
print(df[['SITE NAICS CODE 1', 'SITE NAICS NAME 1']].head())
print("\nUnique NAICS SECTORS 1 after extraction:")
print(df['SITE NAICS NAME 1'].unique())

                                    SITE NAICS CODE 1  \
3   324191 Petroleum Lubricating Oil And Grease Ma...   
18  424690 Other Chemical And Allied Products Merc...   
27                           111930 Sugarcane Farming   
29  424690 Other Chemical And Allied Products Merc...   
30  325199 All Other Basic Organic Chemical Manufa...   

                                    SITE NAICS NAME 1  
3   Petroleum Lubricating Oil And Grease Manufactu...  
18  Other Chemical And Allied Products Merchant Wh...  
27                                  Sugarcane Farming  
29  Other Chemical And Allied Products Merchant Wh...  
30     All Other Basic Organic Chemical Manufacturing  

Unique NAICS SECTORS 1 after extraction:
['Petroleum Lubricating Oil And Grease Manufacturing'
 'Other Chemical And Allied Products Merchant Wholesalers'
 'Sugarcane Farming' 'All Other Basic Organic Chemical Manufacturing'
 'Soap And Other Detergent Manufacturing'
 'Farm Supplies Merchant Wholesalers'
 'Lead Pencil An

In [445]:
naics_sectors = {
    '11': 'Agriculture, Forestry, Fishing and Hunting',
    '21': 'Mining, Quarrying, and Oil and Gas Extraction',
    '22': 'Utilities',
    '23': 'Construction',
    '31': 'Manufacturing', # Covers 31-33
    '32': 'Manufacturing',
    '33': 'Manufacturing',
    '42': 'Wholesale Trade',
    '44': 'Retail Trade', # Covers 44-45
    '45': 'Retail Trade',
    '48': 'Transportation and Warehousing', # Covers 48-49
    '49': 'Transportation and Warehousing',
    '51': 'Information',
    '52': 'Finance and Insurance',
    '53': 'Real Estate and Rental and Leasing',
    '54': 'Professional, Scientific, and Technical Services',
    '55': 'Management of Companies and Enterprises',
    '56': 'Administrative and Support and Waste Management and Remediation Services',
    '61': 'Educational Services',
    '62': 'Health Care and Social Assistance',
    '71': 'Arts, Entertainment, and Recreation',
    '72': 'Accommodation and Food Services',
    '81': 'Other Services (except Public Administration)',
    '92': 'Public Administration'
}

def map_naics_to_sector(naics_id):
  """
  Maps a two-digit NAICS ID to its general sector definition.
  Returns 'Unknown Sector' if the ID is not found.
  """
  if isinstance(naics_id, (int, float)):
      naics_id = str(int(naics_id)) # Convert potential float/int to string

  if isinstance(naics_id, str) and len(naics_id) >= 2:
    two_digit_id = naics_id[:2]
    return naics_sectors.get(two_digit_id, 'Unknown Sector')
  return 'Unknown' # Handle cases where naics_id is not a valid string

df['GENERAL SECTOR NAME'] = df['SITE NAICS ID 1'].apply(map_naics_to_sector)

In [446]:
def get_two_digit_naics(naics_id):
    """
    Extracts the first two digits from a NAICS ID string.
    Returns 'Unknown Sector ID' for invalid or short inputs.
    """
    if isinstance(naics_id, str) and len(naics_id) >= 2 and naics_id.isdigit():
        return naics_id[:2]
    return 'Unknown' # Handle cases like 'Unknown Sector' or shorter strings

df['GENERAL SECTOR ID'] = df['SITE NAICS ID 1'].apply(get_two_digit_naics)


In [447]:
# Display the first few rows with the new column
print(df[['SITE NAICS ID 1', 'GENERAL SECTOR ID']].head())

# Check the unique values in the new column
print("\nUnique GENERAL SECTOR IDs:")
print(df['GENERAL SECTOR ID'].unique())

   SITE NAICS ID 1 GENERAL SECTOR ID
3           324191                32
18          424690                42
27          111930                11
29          424690                42
30          325199                32

Unique GENERAL SECTOR IDs:
['32' '42' '11' '33' '45' '21' '31' '55' 'Unknown' '52' '56' '49' '48'
 '54' '22' '81' '23']


In [448]:
# Display the first few rows with the new column
print(df[['SITE NAICS ID 1', 'GENERAL SECTOR NAME']].head())

# Check the distribution of the new column
print("\nDistribution of GENERAL SECTOR NAME:")
print(df['GENERAL SECTOR NAME'].value_counts())

   SITE NAICS ID 1                         GENERAL SECTOR NAME
3           324191                               Manufacturing
18          424690                             Wholesale Trade
27          111930  Agriculture, Forestry, Fishing and Hunting
29          424690                             Wholesale Trade
30          325199                               Manufacturing

Distribution of GENERAL SECTOR NAME:
GENERAL SECTOR NAME
Manufacturing                                                               37713
Wholesale Trade                                                             11645
Mining, Quarrying, and Oil and Gas Extraction                                1454
Utilities                                                                    1020
Transportation and Warehousing                                                306
Retail Trade                                                                  286
Unknown Sector                                                          

### Membuat Versi Data Frame dengan One Hot Encoding

#### Copy dataset

In [449]:
# prompt: copy df to df_one_hot

df_one_hot = df.copy()

In [450]:
df_one_hot.columns

Index(['CHEMICAL ID', 'CHEMICAL NAME', 'CHEMICAL ID TYPE', 'RECYCLED',
       'PHYSICAL FORM(S)', '2019 DOMESTIC PV', '2019 IMPORT PV', '2019 PV',
       '2018 PV', '2017 PV', '2016 PV', 'SITE LATITUDE', 'SITE LONGITUDE',
       'SITE CITY', 'SITE COUNTY / PARISH', 'SITE STATE', 'SITE POSTAL CODE',
       'SITE NAICS CODE 1', 'SITE NAICS ACTIVITY 1', 'SITE NAICS CODE 2',
       'SITE NAICS ACTIVITY 2', 'SITE NAICS CODE 3', 'SITE NAICS ACTIVITY 3',
       'WORKERS CODE', 'WORKERS', 'MAX CONC CODE', 'MAXIMUM CONCENTRATION',
       'DATA CONSUMER', 'DATA INDUSTRIAL', 'WORKERS MEDIAN',
       'CHEMICAL NAME SUMMARY', 'SITE NAICS ID 1', 'SITE NAICS NAME 1',
       'GENERAL SECTOR NAME', 'GENERAL SECTOR ID'],
      dtype='object')

#### One hot encoding Physcial form(s)

In [451]:
df['PHYSICAL FORM(S)'].unique()

array(['Liquid', 'Dry Powder', 'Unknown', 'NKRA',
       'Water or Solvent Wet Solid', 'Pellets or Large Crystals',
       'Pellets or Large Crystals, Liquid', 'Other Solid',
       'Liquid, Other Solid', 'Dry Powder, Liquid',
       'Dry Powder, Pellets or Large Crystals',
       'Dry Powder, Liquid, Other Solid', 'Gas or Vapor',
       'Gas or Vapor, Liquid, Other Solid',
       'Water or Solvent Wet Solid, Other Solid',
       'Dry Powder, Other Solid',
       'Pellets or Large Crystals, Other Solid',
       'Dry Powder, Water or Solvent Wet Solid',
       'Water or Solvent Wet Solid, Liquid',
       'Dry Powder, Pellets or Large Crystals, Other Solid',
       'Dry Powder, Pellets or Large Crystals, Water or Solvent Wet Solid, Liquid',
       'Gas or Vapor, Other Solid', 'Gas or Vapor, Liquid',
       'Dry Powder, Pellets or Large Crystals, Water or Solvent Wet Solid',
       'Liquid, NKRA',
       'Pellets or Large Crystals, Water or Solvent Wet Solid',
       'Pellets or Large Cry

In [452]:
# prompt: create (7 options - Dry Powder, Pellets or Large Crystals, Water- or Solvent-Wet Solid,
# Other Solid, Gas or Vapor, Liquid, NKRA) for PHYSICAL FORM(S) hot encoding

forms = [
    'Dry Powder',
    'Pellets or Large Crystals',
    'Water or Solvent Wet Solid',
    'Other Solid',
    'Gas or Vapor',
    'Liquid',
    'NKRA'
]

for form in forms:
    df_one_hot['PHYSICAL FORM '+form] = df['PHYSICAL FORM(S)'].apply(lambda x: 1 if form in x else 0)

# Verify the new columns
print(df_one_hot[['PHYSICAL FORM '+form for form in forms]].head())

    PHYSICAL FORM Dry Powder  PHYSICAL FORM Pellets or Large Crystals  \
3                          0                                        0   
18                         1                                        0   
27                         0                                        0   
29                         0                                        0   
30                         0                                        0   

    PHYSICAL FORM Water or Solvent Wet Solid  PHYSICAL FORM Other Solid  \
3                                          0                          0   
18                                         0                          0   
27                                         0                          0   
29                                         0                          0   
30                                         0                          0   

    PHYSICAL FORM Gas or Vapor  PHYSICAL FORM Liquid  PHYSICAL FORM NKRA  
3                            0     

In [453]:
df_one_hot.columns

Index(['CHEMICAL ID', 'CHEMICAL NAME', 'CHEMICAL ID TYPE', 'RECYCLED',
       'PHYSICAL FORM(S)', '2019 DOMESTIC PV', '2019 IMPORT PV', '2019 PV',
       '2018 PV', '2017 PV', '2016 PV', 'SITE LATITUDE', 'SITE LONGITUDE',
       'SITE CITY', 'SITE COUNTY / PARISH', 'SITE STATE', 'SITE POSTAL CODE',
       'SITE NAICS CODE 1', 'SITE NAICS ACTIVITY 1', 'SITE NAICS CODE 2',
       'SITE NAICS ACTIVITY 2', 'SITE NAICS CODE 3', 'SITE NAICS ACTIVITY 3',
       'WORKERS CODE', 'WORKERS', 'MAX CONC CODE', 'MAXIMUM CONCENTRATION',
       'DATA CONSUMER', 'DATA INDUSTRIAL', 'WORKERS MEDIAN',
       'CHEMICAL NAME SUMMARY', 'SITE NAICS ID 1', 'SITE NAICS NAME 1',
       'GENERAL SECTOR NAME', 'GENERAL SECTOR ID', 'PHYSICAL FORM Dry Powder',
       'PHYSICAL FORM Pellets or Large Crystals',
       'PHYSICAL FORM Water or Solvent Wet Solid', 'PHYSICAL FORM Other Solid',
       'PHYSICAL FORM Gas or Vapor', 'PHYSICAL FORM Liquid',
       'PHYSICAL FORM NKRA'],
      dtype='object')

#### One Hot Encoding CHEMICAL ID TYPE

In [454]:
chemical_types = df_one_hot['CHEMICAL ID TYPE'].unique()

In [455]:
chemical_types

array(['Accession Number', 'CASRN'], dtype=object)

In [456]:
for ctype in chemical_types:
    df_one_hot['CHEMICAL ID TYPE '+ctype] = df_one_hot['CHEMICAL ID TYPE'].apply(lambda x: 1 if ctype in x else 0)

In [457]:
df_one_hot.columns

Index(['CHEMICAL ID', 'CHEMICAL NAME', 'CHEMICAL ID TYPE', 'RECYCLED',
       'PHYSICAL FORM(S)', '2019 DOMESTIC PV', '2019 IMPORT PV', '2019 PV',
       '2018 PV', '2017 PV', '2016 PV', 'SITE LATITUDE', 'SITE LONGITUDE',
       'SITE CITY', 'SITE COUNTY / PARISH', 'SITE STATE', 'SITE POSTAL CODE',
       'SITE NAICS CODE 1', 'SITE NAICS ACTIVITY 1', 'SITE NAICS CODE 2',
       'SITE NAICS ACTIVITY 2', 'SITE NAICS CODE 3', 'SITE NAICS ACTIVITY 3',
       'WORKERS CODE', 'WORKERS', 'MAX CONC CODE', 'MAXIMUM CONCENTRATION',
       'DATA CONSUMER', 'DATA INDUSTRIAL', 'WORKERS MEDIAN',
       'CHEMICAL NAME SUMMARY', 'SITE NAICS ID 1', 'SITE NAICS NAME 1',
       'GENERAL SECTOR NAME', 'GENERAL SECTOR ID', 'PHYSICAL FORM Dry Powder',
       'PHYSICAL FORM Pellets or Large Crystals',
       'PHYSICAL FORM Water or Solvent Wet Solid', 'PHYSICAL FORM Other Solid',
       'PHYSICAL FORM Gas or Vapor', 'PHYSICAL FORM Liquid',
       'PHYSICAL FORM NKRA', 'CHEMICAL ID TYPE Accession Number',
   

#### One Hot Encoding RECYCLED

In [458]:
recycled = df_one_hot['RECYCLED'].unique()
recycled = [item for item in recycled if item != 'Unknown']
recycled

['No', 'Yes', 'NKRA']

In [459]:
for r in recycled:
    df_one_hot['RECYCLED '+r] = df_one_hot['RECYCLED'].apply(lambda x: 1 if r in x else 0)

In [460]:
df_one_hot.columns

Index(['CHEMICAL ID', 'CHEMICAL NAME', 'CHEMICAL ID TYPE', 'RECYCLED',
       'PHYSICAL FORM(S)', '2019 DOMESTIC PV', '2019 IMPORT PV', '2019 PV',
       '2018 PV', '2017 PV', '2016 PV', 'SITE LATITUDE', 'SITE LONGITUDE',
       'SITE CITY', 'SITE COUNTY / PARISH', 'SITE STATE', 'SITE POSTAL CODE',
       'SITE NAICS CODE 1', 'SITE NAICS ACTIVITY 1', 'SITE NAICS CODE 2',
       'SITE NAICS ACTIVITY 2', 'SITE NAICS CODE 3', 'SITE NAICS ACTIVITY 3',
       'WORKERS CODE', 'WORKERS', 'MAX CONC CODE', 'MAXIMUM CONCENTRATION',
       'DATA CONSUMER', 'DATA INDUSTRIAL', 'WORKERS MEDIAN',
       'CHEMICAL NAME SUMMARY', 'SITE NAICS ID 1', 'SITE NAICS NAME 1',
       'GENERAL SECTOR NAME', 'GENERAL SECTOR ID', 'PHYSICAL FORM Dry Powder',
       'PHYSICAL FORM Pellets or Large Crystals',
       'PHYSICAL FORM Water or Solvent Wet Solid', 'PHYSICAL FORM Other Solid',
       'PHYSICAL FORM Gas or Vapor', 'PHYSICAL FORM Liquid',
       'PHYSICAL FORM NKRA', 'CHEMICAL ID TYPE Accession Number',
   

#### Change columns to UPPERCASE

In [461]:
# prompt: change all column to uppercase

df_one_hot.columns = df_one_hot.columns.str.upper()
df_one_hot.columns

Index(['CHEMICAL ID', 'CHEMICAL NAME', 'CHEMICAL ID TYPE', 'RECYCLED',
       'PHYSICAL FORM(S)', '2019 DOMESTIC PV', '2019 IMPORT PV', '2019 PV',
       '2018 PV', '2017 PV', '2016 PV', 'SITE LATITUDE', 'SITE LONGITUDE',
       'SITE CITY', 'SITE COUNTY / PARISH', 'SITE STATE', 'SITE POSTAL CODE',
       'SITE NAICS CODE 1', 'SITE NAICS ACTIVITY 1', 'SITE NAICS CODE 2',
       'SITE NAICS ACTIVITY 2', 'SITE NAICS CODE 3', 'SITE NAICS ACTIVITY 3',
       'WORKERS CODE', 'WORKERS', 'MAX CONC CODE', 'MAXIMUM CONCENTRATION',
       'DATA CONSUMER', 'DATA INDUSTRIAL', 'WORKERS MEDIAN',
       'CHEMICAL NAME SUMMARY', 'SITE NAICS ID 1', 'SITE NAICS NAME 1',
       'GENERAL SECTOR NAME', 'GENERAL SECTOR ID', 'PHYSICAL FORM DRY POWDER',
       'PHYSICAL FORM PELLETS OR LARGE CRYSTALS',
       'PHYSICAL FORM WATER OR SOLVENT WET SOLID', 'PHYSICAL FORM OTHER SOLID',
       'PHYSICAL FORM GAS OR VAPOR', 'PHYSICAL FORM LIQUID',
       'PHYSICAL FORM NKRA', 'CHEMICAL ID TYPE ACCESSION NUMBER',
   

### Export Data Frame to CSV

In [462]:
# prompt: i want to export df to csv

df.to_csv('df.csv', index=False)
df_one_hot.to_csv('df_one_hot.csv', index=False)